In [3]:
import random

def demorgan_quiz(rounds=6):
    print("🧠 De Morgan's Laws Quiz")
    print("For each expression, decide if it's ALWAYS true (True) or not always true (False).")
    print("Type True or False and press Enter.\n")

    questions = [
        # Correct laws
        ("not (A and B) == (not A) or (not B)", True),
        ("not (A or B) == (not A) and (not B)", True),
        # Incorrect variants
        ("not (A and B) == (not A) and (not B)", False),
        ("not (A or B) == (not A) or (not B)", False),
        ("not (A) and not (B) == not (A and B)", False),
    ]

    score = 0
    for i in range(1, rounds + 1):
        A = random.choice([True, False])
        B = random.choice([True, False])
        expr, correct = random.choice(questions)

        print(f"Round {i}: A = {A}, B = {B}")
        print(f"Check this statement:\n   {expr}")

        guess = input("Is this always true? (True/False): ").strip()

        # Normalize student input
        guess_bool = guess.lower().startswith("t")

        if guess_bool == correct:
            print("✅ Correct!\n")
            score += 1
        else:
            print(f"❌ Incorrect. The right answer was {correct}.\n")

    print(f"🏁 Final score: {score}/{rounds}")
    if score == rounds:
        print("🎉 Excellent! You really understand De Morgan’s Laws.")
    elif score >= rounds * 0.7:
        print("👍 Good job! Review the tricky ones and try again.")
    else:
        print("📘 Keep practicing — focus on how NOT distributes over AND/OR.")


In [5]:
# Run the quiz
demorgan_quiz(rounds=4)


🧠 De Morgan's Laws Quiz
For each expression, decide if it's ALWAYS true (True) or not always true (False).
Type True or False and press Enter.

Round 1: A = False, B = True
Check this statement:
   not (A or B) == (not A) or (not B)
✅ Correct!

Round 2: A = True, B = False
Check this statement:
   not (A or B) == (not A) and (not B)
✅ Correct!

Round 3: A = True, B = False
Check this statement:
   not (A and B) == (not A) or (not B)
✅ Correct!

Round 4: A = True, B = True
Check this statement:
   not (A and B) == (not A) or (not B)
✅ Correct!

🏁 Final score: 4/4
🎉 Excellent! You really understand De Morgan’s Laws.


## Random Arguments

In [12]:
import itertools
import random

# --- 1. Random literal (with at most one 'not') ---
def random_literal(variables):
    var = random.choice(variables)
    return var if random.random() > 0.5 else f"not {var}"

# --- 2. Build a random formula recursively ---
def random_formula(variables, depth=2):
    if depth == 0:
        return random_literal(variables)
    left = random_formula(variables, depth - 1)
    right = random_formula(variables, depth - 1)
    op = random.choice(["and", "or", "->", "<->"])
    if op == "->":
        return f"(not {left} or {right})"  # p -> q ≡ ¬p ∨ q
    elif op == "<->":
        return f"(({left} and {right}) or (not {left} and not {right}))"
    else:
        return f"({left} {op} {right})"

# --- 3. Evaluate a formula for a given row of truth values ---
def eval_formula(formula, values):
    return eval(formula, {}, values)

# --- 4. Build truth table ---
def truth_table(variables, formulas):
    table = []
    for combo in itertools.product([False, True], repeat=len(variables)):
        vals = dict(zip(variables, combo))
        row = {v: vals[v] for v in variables}
        for label, f in formulas.items():
            row[label] = eval_formula(f, vals)
        table.append(row)
    return table

# --- 5. Generate a random argument ---
def random_argument(variables=["p", "q", "r"], num_premises=2):
    premises = [random_formula(variables) for _ in range(num_premises)]
    conclusion = random_formula(variables)
    return premises, conclusion

# --- 6. Check argument validity ---
def check_argument(premises, conclusion, variables):
    formulas = {f"P{i+1}": prem for i, prem in enumerate(premises)}
    formulas["Conclusion"] = conclusion
    table = truth_table(variables, formulas)
    valid = True
    counterexamples = []
    for row in table:
        if all(row[f"P{i+1}"] for i in range(len(premises))) and not row["Conclusion"]:
            valid = False
            counterexamples.append(row)
    return valid, counterexamples

# --- 7. Pretty-print formulas with logical symbols ---
def pretty(formula):
    # Replace carefully: longer substrings first
    formula = formula.replace("<->", "↔")
    formula = formula.replace("->", "→")
    formula = formula.replace(" and ", " ∧ ")
    formula = formula.replace(" or ", " ∨ ")
    formula = formula.replace("not ", "¬")
    return formula


In [15]:

# --- Demo run ---
if __name__ == "__main__":
    premises, conclusion = random_argument()
    print("🧠 Randomly generated argument:")
    for i, p in enumerate(premises, 1):
        print(f"  Premise {i}: {pretty(p)}")
    print(f"  Conclusion: {pretty(conclusion)}")

    valid, counterexamples = check_argument(premises, conclusion, ["p", "q", "r"])

    print("\n📊 Result:")
    if valid:
        print("✅ The argument is VALID (no counterexamples).")
    else:
        print("❌ The argument is INVALID. Counterexamples:")
        for c in counterexamples:
            print("   " + ", ".join(f"{k}={v}" for k, v in c.items()))


🧠 Randomly generated argument:
  Premise 1: (((¬p ∧ ¬p) ∨ (¬¬p ∧ ¬¬p)) ∧ (p ∧ r))
  Premise 2: ((p ∨ ¬q) ∧ ((q ∧ p) ∨ (¬q ∧ ¬p)))
  Conclusion: ((¬r ∨ ¬p) ∧ (r ∧ r))

📊 Result:
❌ The argument is INVALID. Counterexamples:
   p=True, q=True, r=True, P1=True, P2=True, Conclusion=False


In [13]:

# --- Demo run ---
premises, conclusion = random_argument()
print("🧠 Randomly generated argument:")
for i, p in enumerate(premises, 1):
    print(f"  Premise {i}: {pretty(p)}")
print(f"  Conclusion: {pretty(conclusion)}")

valid, counterexamples = check_argument(premises, conclusion, ["p", "q", "r"])

print("\n📊 Result:")
if valid:
    print("✅ The argument is VALID (no counterexamples).")
else:
    print("❌ The argument is INVALID. Counterexamples:")
    for c in counterexamples:
        print("   " + ", ".join(f"{k}={v}" for k, v in c.items()))


🧠 Randomly generated argument:
  Premise 1: (¬(¬¬p ∨ p) ∨ ((¬q ∧ p) ∨ (¬¬q ∧ ¬p)))
  Premise 2: ((¬r ∨ ¬p) ∨ (¬¬q ∨ p))
  Conclusion: ((¬r ∨ p) ∧ (r ∨ ¬q))

📊 Result:
❌ The argument is INVALID. Counterexamples:
   p=False, q=False, r=True, P1=True, P2=True, Conclusion=False
   p=False, q=True, r=False, P1=True, P2=True, Conclusion=False
   p=False, q=True, r=True, P1=True, P2=True, Conclusion=False
